In [4]:
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수를 불러옵니다.
load_dotenv()

# 환경 변수 읽기
openai_api_key = os.getenv("OPENAI_API_KEY")


# OpenAI 라이브러리가 환경 변수에서 키를 읽을 수 있도록 설정
if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key

print("API Key configured:", "OPENAI_API_KEY" in os.environ)

from openai import OpenAI
client = OpenAI()

API Key configured: True


In [8]:
response = client.chat.completions.create(
    model="gpt-4o-mini",  # 또는 'gpt-3.5-turbo' 등
    messages=[
        {"role": "user", "content": "파이썬에 대해 알려줘"}
    ]
)

In [10]:
message = {'role':'user',"content":'안녕하세요, 오늘 날씨는 어떤가요?'}
response = client.chat.completions.create(
    model ='gpt-4o-mini',
    messages=[message]
)

assistant_reply = response.choices[0].message.content
print("ChatGPT 답변:", assistant_reply)

ChatGPT 답변: 안녕하세요! 현재 날씨 정보는 실시간으로 제공할 수 없지만, 귀하가 위치한 지역의 날씨를 확인하려면 날씨 앱이나 웹사이트를 참고해 주세요. 도움이 필요하신 다른 사항이 있다면 말씀해 주세요!


# 출력 형식 지정

In [ ]:
message = {'role':'user',"content":'안녕하세요, 오늘 서울 날씨를 json 형태로 제공해. 키는 weather, temperature 로 설정해'}
response = client.chat.completions.create(
    model ='gpt-4o-mini',
    messages=[message], response_format={'type':'json_object'} # 출력형식 지정
)

assistant_reply = response.choices[0].message.content
print("ChatGPT 답변:", assistant_reply)

ChatGPT 답변: {"weather": "맑음", "temperature": "20도"}


In [15]:
import json
data = json.loads(assistant_reply)
data.keys()

dict_keys(['weather', 'temperature'])

In [16]:
# 전라북도 날씨 json으로 불러오기
message = {'role':'user',"content":'안녕하세요, 오늘 전라북도의 도시 5개를 선정해서 날씨를 json 형태로 제공해. 키는 weather, temperature 로 설정해'}
response = client.chat.completions.create(
    model ='gpt-4o-mini',
    messages=[message], response_format={'type':'json_object'} # 출력형식 지정
)

assistant_reply = response.choices[0].message.content
print("ChatGPT 답변:", assistant_reply)

ChatGPT 답변: 
{
    "전라북도_도시_날씨": [
        {
            "도시": "전주",
            "weather": "맑음",
            "temperature": "25도"
        },
        {
            "도시": "군산",
            "weather": "구름 많음",
            "temperature": "22도"
        },
        {
            "도시": "익산",
            "weather": "비",
            "temperature": "20도"
        },
        {
            "도시": "남원",
            "weather": "흐림",
            "temperature": "23도"
        },
        {
            "도시": "정읍",
            "weather": "맑음",
            "temperature": "24도"
        }
    ]
}


In [20]:
weather_data = json.loads(assistant_reply)

for entry in weather_data["전라북도_도시_날씨"]:
    city = entry["도시"]
    weather = entry["weather"]
    temperature = entry["temperature"]
    print(f"{city}의 날씨는 현재 {weather}이고, 온도는 {temperature}입니다.")

전주의 날씨는 현재 맑음이고, 온도는 25도입니다.
군산의 날씨는 현재 구름 많음이고, 온도는 22도입니다.
익산의 날씨는 현재 비이고, 온도는 20도입니다.
남원의 날씨는 현재 흐림이고, 온도는 23도입니다.
정읍의 날씨는 현재 맑음이고, 온도는 24도입니다.


# CoT 프롬프트

In [22]:
question = '철수는 사과 10개를 가지고 있었는데, 그 중 3개를 먹은 후 5개를 더 구입했습니다. 철수가 현재 가지고 있는 사과는 몇 개일까요?'
response = client.chat.completions.create(
    messages= [{'role':'user','content':question}]
    , model= 'gpt-4o-mini',
    temperature=0
)

print('COT 적용 전:',response.choices[0].message.content)

COT 적용 전: 철수는 처음에 사과 10개를 가지고 있었습니다. 그 중 3개를 먹었으므로 남은 사과는 10 - 3 = 7개입니다. 이후 5개를 더 구입했으므로 현재 가지고 있는 사과는 7 + 5 = 12개입니다. 

따라서 철수가 현재 가지고 있는 사과는 12개입니다.


In [23]:
cot_prompt= '(생각을 단계별로 진행해)\n' + question
res = client.chat.completions.create(
    model="gpt-4o-mini",
  messages=[
      {'role':'user', 'content':cot_prompt}
  ],
    temperature=0
)
print('cot 적용전: ', res.choices[0].message.content)

cot 적용전:  단계별로 생각해보겠습니다.

1. 철수가 처음 가지고 있던 사과의 개수: 10개
2. 철수가 먹은 사과의 개수: 3개
3. 철수가 먹은 후 남은 사과의 개수: 10개 - 3개 = 7개
4. 철수가 추가로 구입한 사과의 개수: 5개
5. 철수가 현재 가지고 있는 사과의 총 개수: 7개 + 5개 = 12개

따라서, 철수가 현재 가지고 있는 사과는 12개입니다.


In [ ]:
question = '어떤 수를 3배한 결과에 4를 더하면 19가 됩니다. 그 수는 무엇알까요? 단계별로 풀어보세요.'
cot_prompt= '(생각을 단계별로 진행해)\n' + question
res = client.chat.completions.create(
    model="gpt-4o-mini",
  messages=[
      {'role':'user', 'content':cot_prompt}
  ],
    temperature=0 # 창의적이지 않고 고지식하게 답변
)
print('cot 적용전: ', res.choices[0].message.content)

cot 적용전:  주어진 문제를 단계별로 풀어보겠습니다.

1. **문제 이해하기**: 어떤 수를 \( x \)라고 할 때, 이 수를 3배한 결과에 4를 더하면 19가 된다는 것입니다. 이를 수식으로 표현하면 다음과 같습니다.
   \[
   3x + 4 = 19
   \]

2. **방정식 정리하기**: 이제 이 방정식을 풀어보겠습니다. 먼저, 양쪽에서 4를 빼줍니다.
   \[
   3x + 4 - 4 = 19 - 4
   \]
   \[
   3x = 15
   \]

3. **x 구하기**: 이제 양쪽을 3으로 나누어 \( x \)의 값을 구합니다.
   \[
   x = \frac{15}{3}
   \]
   \[
   x = 5
   \]

4. **정답 확인하기**: 구한 \( x \)의 값이 맞는지 확인해보겠습니다. \( x = 5 \)일 때, 3배한 결과에 4를 더해보면:
   \[
   3 \times 5 + 4 = 15 + 4 = 19
   \]
   맞습니다.

따라서, 그 수는 **5**입니다.


검증이 가능하게 프롬프트가 짜여졌다.

# self consistency 기법

In [27]:
messages = [
    {'role': 'user', 'content': question}
]

answers = []

for i in range(5):
    response = client.chat.completions.create(
        messages=messages,
        model='gpt-4o-mini',
        temperature=1.0
    )
    answer = response.choices[0].message.content.strip()
    answers.append(answer)
    print(f'응답 {i+1}: {answer}')

응답 1: 주어진 문제를 단계별로 해결해 보겠습니다.

1. 문제를 수식으로 표현해 보겠습니다.
   - 어떤 수를 \( x \)라고 하겠습니다.
   - 그러면 "3배한 결과에 4를 더하면 19가 된다"는 내용을 수식으로 표현하면 다음과 같습니다:
   \[
   3x + 4 = 19
   \]

2. 이제 이 수식을 풀어 보겠습니다.
   - 먼저 4를 양변에서 빼줍니다:
   \[
   3x + 4 - 4 = 19 - 4
   \]
   \[
   3x = 15
   \]

3. 다음으로, 양변을 3으로 나누어 \( x \)를 구합니다:
   \[
   \frac{3x}{3} = \frac{15}{3}
   \]
   \[
   x = 5
   \]

4. 따라서, 그 수는 \( \boxed{5} \)입니다.

5. 확인을 위해 원래 문제의 조건을 다시 확인해 보겠습니다.
   - \( x = 5 \)일 때, \( 3 \times 5 + 4 = 15 + 4 = 19 \)가 됩니다. 따라서 조건이 맞습니다.

결론적으로, 그 수는 \( 5 \)입니다.
응답 2: 주어진 문제를 단계별로 풀어보겠습니다.

1. **문제 이해하기**: 어떤 수를 \( x \)라고 하면, 이 수를 3배한 결과에 4를 더한 것이 19가 된다는 것입니다. 이를 식으로 표현해보면:
   \[
   3x + 4 = 19
   \]

2. **식 정리하기**: 이제 이 식에서 \( x \)를 구하기 위해 식을 변형해 보겠습니다. 먼저, 4를 왼쪽으로 이동시킵니다.
   \[
   3x = 19 - 4
   \]
   \[
   3x = 15
   \]

3. **\( x \) 구하기**: \( x \)를 구하기 위해 양변을 3으로 나눕니다.
   \[
   x = \frac{15}{3}
   \]
   \[
   x = 5
   \]

4. **결과 확인하기**: \( x = 5 \)가 맞는지 확인하기 위해 원래의 식에 대입해 봅시다.
   \[
   3(5) + 4 = 15 + 4 = 

In [32]:
final_answer = [ans.split()[-1] for ans in answers]
import collections
counter = collections.Counter(final_answer)
final_ans = counter.most_common(1)[0][0]
print(f'최종 답:{final_ans}')

최종 답:**5**입니다.


# Reflexion

- 자기 피드백 반복 프롬프트 설계

In [33]:
question = '7의 2승에 5를 곱합 값은 무엇인가?'
response1 = client.chat.completions.create(
    model="gpt-4o-mini",
  messages=[
      {'role':'user', 'content':question}
  ],
    temperature=0
)
answer = response1.choices[0].message.content.strip() # 모델이 출력한 정답
correct_answer = 245
if str(correct_answer) in answer:
    feedback = '정답입니다.'
    need_retry = False
else:
    feedback = '틀렸어요. 힌트는 7의 제곱값을 정확히 구한 다음 곱하세요.'
    need_retry = True
    
print(feedback)

정답입니다.


In [38]:
question = '7의 2승에 5를 곱합 값은 무엇인가?'
response1 = client.chat.completions.create(
    model="gpt-3.5-turbo",
  messages=[
      {'role':'user', 'content':question}
  ],
    temperature=0
)
answer = response1.choices[0].message.content.strip() # 모델이 출력한 정답
correct_answer = 245
if str(correct_answer) in answer:
    feedback = '정답입니다.'
    need_retry = False
else:
    feedback = '틀렸어요. 힌트는 7의 제곱값을 정확히 구한 다음 곱하세요.'
    need_retry = True
    
print(feedback)

정답입니다.


In [ ]:
# 틀린모델은? 2차 답변 생성.
if need_retry:
    retry_prompt = f'이전 답변:{answer}, 피드백:{feedback}\n 다시 답을 구하세요'
    response2 = client.chat.completions.create(
    model="gpt-3.5-turbo",
  messages=[
      {'role':'user', 'content':question}
  ],
    temperature=0
)
answer2 = response2.choices[0].message.content.strip() # 모델이 출력한 응답
print(answer2)

In [40]:
prompt = '기후 변화의 원인과 해결 방안에 대해 간략한 에세이를 작성하시오'
res2 = client.chat.completions.create(
    model="gpt-4o-mini",
  messages=[
      {'role':'user', 'content':prompt}
  ],
    temperature=0
)
print(res2.choices[0].message.content)

### 기후 변화의 원인과 해결 방안

기후 변화는 현대 사회가 직면한 가장 심각한 환경 문제 중 하나로, 지구의 평균 기온 상승, 해수면 상승, 극단적인 기상 현상 증가 등 다양한 형태로 나타나고 있다. 이러한 기후 변화의 주된 원인은 인간 활동에 의해 발생하는 온실가스의 배출이다. 특히, 화석 연료의 연소, 산업 활동, 농업 및 임업 등에서 발생하는 이산화탄소(CO2), 메탄(CH4), 아산화질소(N2O) 등의 온실가스가 대기 중에 축적되어 지구의 온도를 상승시키고 있다.

기후 변화의 해결 방안은 여러 가지가 있지만, 가장 효과적인 방법은 온실가스 배출을 줄이는 것이다. 이를 위해 첫째, 재생 가능 에너지의 사용을 확대해야 한다. 태양광, 풍력, 수력 등 청정 에너지원으로의 전환은 화석 연료 의존도를 줄이고, 온실가스 배출을 감소시킬 수 있다. 둘째, 에너지 효율성을 높이는 기술을 개발하고 보급해야 한다. 건물, 교통수단, 산업 공정에서 에너지를 절약하는 방법을 도입함으로써 전체적인 에너지 소비를 줄일 수 있다.

셋째, 지속 가능한 농업과 임업 practices를 채택해야 한다. 이는 토양의 탄소 저장 능력을 높이고, 생물 다양성을 보호하며, 기후 변화에 대한 적응력을 강화하는 데 기여할 수 있다. 넷째, 개인과 지역 사회의 인식 개선이 필요하다. 기후 변화의 심각성을 알리고, 개인의 행동 변화(예: 대중교통 이용, 재활용, 에너지 절약 등)를 유도하는 교육과 캠페인이 중요하다.

마지막으로, 국제적인 협력이 필수적이다. 기후 변화는 국경을 초월하는 문제이므로, 각국이 협력하여 공동의 목표를 설정하고, 기술과 자원을 공유하는 것이 중요하다. 파리 협정과 같은 국제적인 합의는 이러한 협력을 촉진하는 데 중요한 역할을 한다.

결론적으로, 기후 변화는 인류의 생존과 미래에 중대한 영향을 미치는 문제이다. 이를 해결하기 위해서는 개인, 기업, 정부가 함께 노력하여 지속 가능한 사회를 만들어 나가야 한다. 기후 변화에 대한 적극적인 대응은 단순히 환경 보호를 

In [41]:
# 피드백 작성 (여기서는 사람이 수동으로 평가하여 문자열 작성한다고 가정)
feedback = (
    "피드백:\n"
    " - 서론에서 주제 소개가 부족합니다.\n"
    " - 원인에 대한 설명이 모호하며 구체적 예시와 근거를 제시할 통계자료가 없습니다.\n"
    " - 해결 방안을 최소 10가지 이상 제시해주세요.\n"
    "위 사항을 반영하여 에세이를 수정해주세요."
)
# 개선 프롬프트 구성
retry_prompt = res2.choices[0].message.content + "\n\n" + feedback
response_essay2 = client.chat.completions.create(
    messages=[
        {"role": "user", "content": retry_prompt}
    ],
    model="gpt-4o-mini",
    temperature=0
)
essay2 = response_essay2.choices[0].message.content
print("2차 에세이:\n", essay2)

2차 에세이:
 ### 기후 변화의 원인과 해결 방안

기후 변화는 현대 사회가 직면한 가장 심각한 환경 문제 중 하나로, 지구의 평균 기온 상승, 해수면 상승, 극단적인 기상 현상 증가 등 다양한 형태로 나타나고 있다. 이러한 변화는 인류의 생존과 미래에 중대한 영향을 미치며, 이를 해결하기 위한 노력이 시급하다. 본 에세이에서는 기후 변화의 주된 원인과 그 해결 방안을 구체적으로 살펴보겠다.

기후 변화의 주된 원인은 인간 활동에 의해 발생하는 온실가스의 배출이다. 특히, 화석 연료의 연소, 산업 활동, 농업 및 임업 등에서 발생하는 이산화탄소(CO2), 메탄(CH4), 아산화질소(N2O) 등의 온실가스가 대기 중에 축적되어 지구의 온도를 상승시키고 있다. 예를 들어, 2021년 기준으로 전 세계 이산화탄소 배출량은 약 36.4 기가톤에 달하며, 이는 2010년 대비 약 60% 증가한 수치이다. 이러한 배출량 증가는 기후 변화의 가속화에 직접적인 영향을 미치고 있다.

기후 변화의 해결 방안은 여러 가지가 있지만, 가장 효과적인 방법은 온실가스 배출을 줄이는 것이다. 이를 위해 다음과 같은 10가지 이상의 구체적인 방안을 제시할 수 있다.

1. **재생 가능 에너지의 사용 확대**: 태양광, 풍력, 수력 등 청정 에너지원으로의 전환은 화석 연료 의존도를 줄이고, 온실가스 배출을 감소시킬 수 있다.

2. **에너지 효율성 향상**: 건물, 교통수단, 산업 공정에서 에너지를 절약하는 기술을 개발하고 보급하여 전체적인 에너지 소비를 줄인다.

3. **지속 가능한 농업 및 임업 실천**: 토양의 탄소 저장 능력을 높이고, 생물 다양성을 보호하며, 기후 변화에 대한 적응력을 강화하는 농업 및 임업 방법을 채택한다.

4. **전기차 및 대중교통 이용 촉진**: 전기차의 보급을 늘리고, 대중교통 시스템을 개선하여 개인 차량 사용을 줄인다.

5. **탄소 가격제 도입**: 탄소 배출에 대한 가격을 책정하여 기업과 개인이 온실가스를 줄이도록 유도한다.

6. **